In [ ]:
# retrieve the ISIC dataset stuff
!wget https://isic-challenge-data.s3.amazonaws.com/2024/ISIC_2024_Training_Input.zip
!unzip ISIC_2024_Training_Input.zip

!wget https://isic-challenge-data.s3.amazonaws.com/2024/ISIC_2024_Training_Supplement.csv
!wget https://isic-challenge-data.s3.amazonaws.com/2024/ISIC_2024_Training_GroundTruth.csv

In [6]:
# params
FREEZE = False # whether or not to freeze the weights
EPOCHS = 5 # number of training epochs
DUPE = True
!#pip install --upgrade s3fs
#!pip install peft

In [2]:
# setup the model to be transfer learned
import torch
from peft import LoraConfig, get_peft_model
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from sklearn.model_selection import train_test_split

/Users/forestzhang/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
transform = transforms.Compose([
    transforms.ToTensor()
])

class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith(('png', 'jpg', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [4]:


# Define a transform to convert images to tensors
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load the dataset
dataset = CustomImageDataset(root_dir='./ISIC_2024_Training_Input/', transform=transform)
loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=0)

# Calculate mean and standard deviation
mean = 0.0
std = 0.0
nb_samples = 0

for data in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

# Print the calculated mean and standard deviation
print(f'Mean: {mean}')
print(f'Std: {std}')

RuntimeError: stack expects each tensor to be equal size, but got [3, 127, 127] at entry 0 and [3, 159, 159] at entry 1

In [5]:

df = pd.read_csv('ISIC_2024_Training_GroundTruth.csv')
print(f"num malignant: {sum(df['malignant'])}")

# downsample for time
df = df.sample(frac=.2, random_state=42)

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['malignant'], random_state=42)

if DUPE:
    malig_df = train_df[train_df['malignant'] == 1]
    malig_df = pd.concat([malig_df]*100, ignore_index=True)

    train_df = pd.concat([train_df, malig_df], ignore_index=True)

    #malig_df = val_df[val_df['malignant'] == 1]
    #malig_df = pd.concat([malig_df]*100, ignore_index=True)

    #val_df = pd.concat([val_df, malig_df], ignore_index=True)

train_df.to_csv('train_labels.csv', index=False)
val_df.to_csv('val_labels.csv', index=False)

print(f"Validation - Num malignant: {val_df['malignant'].sum()}")
print(f"Validation - Num benign: {len(val_df) - val_df['malignant'].sum()}")
print(f"Training - Num malignant: {train_df['malignant'].sum()}")
print(f"Training - Num benign: {len(train_df) - train_df['malignant'].sum()}")

class ISICDataset(Dataset):
    def __init__(self, csv_file, img_dir, non_malignant_transform=None, malignant_transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.non_malignant_transform = non_malignant_transform
        self.malignant_transform = malignant_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0] + '.jpg')

        image = Image.open(img_path).convert('RGB')

        if self.df.iloc[idx, 1] == 0 and self.non_malignant_transform:
            image = self.non_malignant_transform(image)
        elif self.df.iloc[idx, 1] == 1 and self.malignant_transform:
            image = self.malignant_transform(image)

        return image, int(self.df.iloc[idx, 1]), self.df.iloc[idx, 0]



transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet means
                         std=[0.229, 0.224, 0.225])    # ImageNet stds
])

malignant_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    #transforms.RandomRotation(degrees=30),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_dataset = ISICDataset(
    csv_file='val_labels.csv',
    img_dir='./ISIC_2024_Training_Input/',
    non_malignant_transform=transform,
    malignant_transform=transform
)

train_dataset = ISICDataset(
    csv_file='train_labels.csv',
    img_dir='./ISIC_2024_Training_Input/',
    non_malignant_transform=transform,
    malignant_transform=malignant_transform
)

dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

print(f"Training set size: {len(dataloader.dataset)}")
print(f"Validation set size: {len(valloader.dataset)}")
#malig = sum([x[1] for x in dataloader.dataset])


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {'cuda' if torch.cuda.is_available() else 'cpu'}")

num malignant: 393.0
Validation - Num malignant: 15.0
Validation - Num benign: 16028.0
Training - Num malignant: 5959.0
Training - Num benign: 64110.0
Training set size: 70069
Validation set size: 16043
Training on cpu


In [ ]:
#LORA with dropout
import time
from sklearn.metrics import roc_auc_score

resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)



if FREEZE:
  for param in resnet.parameters():
    param.requires_grad = False

# set to 1 for now (either malignant or not, can change based on metadata later)
resnet.fc = nn.Linear(resnet.fc.in_features, 1)

lora_config = LoraConfig(
    r=8,  # Low-rank adaptation size
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=[ "fc",             # Final classification layer
        "layer2.0.conv1", # First conv in layer2
        "layer3.1.conv2",]  # Apply LoRA to the fully connected layer
)

resnet = get_peft_model(resnet, lora_config)


EPOCH: 1


KeyboardInterrupt: 

In [11]:
#DROPOUT only
class ResNetWithDropout(nn.Module):
    def __init__(self, dropout_prob=0.1):
        super().__init__()
        self.backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        
        # Add spatial dropout between blocks
        self.backbone.layer2.add_module("dropout", nn.Dropout2d(dropout_prob))
        self.backbone.layer3.add_module("dropout", nn.Dropout2d(dropout_prob))
        
        # Final layer dropout
        self.backbone.fc = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(self.backbone.fc.in_features, 1)
        )
    
    def forward(self, x):
        return self.backbone(x)

resnet = ResNetWithDropout(dropout_prob=0.15)



In [ ]:

class EnhancedDisparityLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([100.0]))
        
    def forward(self, outputs, targets, skin_tones):
        # Base loss
        base_loss = self.base_loss(outputs, targets.float())
        
        # Calculate performance disparities
        pred_probs = torch.sigmoid(outputs)
        fprs, fnrs = [], []
        
        for tone in torch.unique(skin_tones):
            mask = (skin_tones == tone)
            if mask.sum() > 0:
                tn, fp, fn, tp = self._confusion_matrix(pred_probs[mask], targets[mask])
                fprs.append(fp / (fp + tn + 1e-6))  # Avoid division by zero
                fnrs.append(fn / (fn + tp + 1e-6))
        
        # Disparity penalties
        fpr_disparity = torch.std(torch.stack(fprs))
        fnr_disparity = torch.std(torch.stack(fnrs))
        
        # Weighted combination
        return base_loss + 0.4*fpr_disparity + 0.6*fnr_disparity  # FNR disparity more critical for cancer

    def _confusion_matrix(self, probs, targets):
        preds = (probs > 0.5).float()
        tp = (preds * targets).sum()
        fp = (preds * (1 - targets)).sum()
        tn = ((1 - preds) * (1 - targets)).sum()
        fn = ((1 - preds) * targets).sum()
        return tn, fp, fn, tp

In [12]:

optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=1e-4)

#criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([(len(dataloader.dataset) - malig)/malig]).to(device))
criterion = EnhancedDisparityLoss().to(device)

# switch to this if we're doing more than malignant/not-malignant
#criterion = nn.CrossEntropyLoss()

resnet.to(device)

start_time = time.time()
for epoch in range(EPOCHS):
    resnet.train()
    avgloss = 0.0
    print(f"EPOCH: {epoch + 1}")
    for images, labels, _ in dataloader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        avgloss += loss.item() * images.size(0)
    avgloss /= len(dataloader.dataset)

    resnet.eval()
    val_loss = 0.0
    total = 0
    false_negative = 0
    false_positive = 0
    true_negative = 0
    true_positive = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels, _ in valloader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

            outputs = resnet(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            preds = torch.sigmoid(outputs)
            preds = (preds > 0.5).float()  # threshold to get 0 or 1
            for p, l in zip(preds, labels):
                if p == 0 and l == 0:
                    true_negative += 1
                elif p == 0 and l == 1:
                    false_negative += 1
                elif p == 1 and l == 0:
                    false_positive += 1
                elif p == 1 and l == 1:
                    true_positive += 1
            total += labels.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    avg_val_loss = val_loss / len(valloader.dataset)
    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Train Loss: {avgloss:.4f} "
          f"Val Loss: {avg_val_loss:.4f} \n"
          f"Val True Negative: {true_negative} "
          f"Val False Negative: {false_negative}\n"
          f"Val True Positive: {true_positive} "
          f"Val False Positive: {false_positive}\n"
          f" Val Accuracy: {(true_positive+true_negative)/(total):.4f}\n"
          f" Val AUC: {roc_auc_score(all_labels, all_preds):.4f}")
    
print(f"Training time: {time.time() - start_time:.2f}s")

EPOCH: 1


KeyboardInterrupt: 